In [1]:
import sys
sys.path.append('../..')
import numpy as np
import matplotlib.pyplot as plt

# PyTorch
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import cuda
from torch.autograd import Variable
from torchvision.utils import make_grid
from torchvision.models.mobilenet import mobilenet_v2

# DLBio and own scripts
from DLBio.pytorch_helpers import get_device
import ds_ear_siamese
import transforms_data as td
from helpers import cuda_conv
import metrics as M
from siamese_network_train import Training
from ContrastiveLossFunction import ContrastiveLoss
from NN_Siamese import SiameseNetwork

from PIL import Image
import glob

In [2]:
class Config():
    DEVICE = get_device()
    DATASET_DIR = '../dataset/'
    MODEL_DIR = './models/model_MN_allunfreezed.pt'
    RESIZE_SMALL = False
    DATABASE_FOLDER = './embeddings/'

In [3]:
model = torch.load(Config.MODEL_DIR)
model.to(Config.DEVICE)
preprocess = td.transforms_siamese_verification( td.get_resize(Config.RESIZE_SMALL) )

In [4]:
def image_pipeline(input_, preprocess):
    input_ = input_.convert("L")
    input_ = preprocess(input_)
    input_ = input_.reshape(-1, td.get_resize(Config.RESIZE_SMALL)[0], td.get_resize(Config.RESIZE_SMALL)[1], 1)
    input_ = input_.permute(3, 0, 1, 2)   
    if cuda.is_available():
        return input_.type('torch.cuda.FloatTensor')
    else:
        return input_.type('torch.FloatTensor')

In [6]:
label_list = []
test = []
torch.cuda.synchronize()
for label in os.listdir(Config.DATASET_DIR):
    label_list.append(label)
    image_list = []
    for filename in glob.glob(Config.DATASET_DIR+label+'/*'): 
        img = Image.open(filename)
        img_processed = image_pipeline(img,preprocess)
        image_list.append(img_processed)
    
    embeddings = np.array([model(Variable(i)).cpu() for i in image_list])
    
    np.save(Config.DATABASE_FOLDER+label+'.npy', embeddings)

    embeddings = []

    